# Create DTA Templates

Creates **DTA Templates** from approved DTAs, scoped by **(vendor, data_stream)** combination.

**Operation:** `CREATE_DTA_TEMPLATE`

**What it does:**
1. Groups all approved DTAs by (data_provider_name, data_stream_type)
2. For each vendor+stream group:
   - Finds the current DTA Template for that vendor+stream (if exists)
   - Identifies new approved DTAs not yet included in any template
   - Merges new DTAs with existing template records
   - Creates a new DTA_TEMPLATE version (e.g., "1.0" for first, "2.0" for next)
3. Closes previous template for that vendor+stream using SCD Type 2
4. Updates DTA `status` to `PROMOTED` for all merged DTAs
5. Tracks which DTAs were included in each template (`included_dta_ids`)

**Incremental Merge Logic:**
- Template v1.0 for (Vendor_A, EDC_Data) has 85 records from DTA001, DTA002
- DTA003 (same vendor+stream) is approved with 10 new records
- **Result:** Template v2.0 will have 85 + 10 = 95 records (minus duplicates)
- Template for other vendor+stream combinations are NOT affected

**Template Scoping:**
- Each (vendor, data_stream) combination has its own version sequence
- Templates are independent per vendor+stream

**Parameters:**
- `merge_strategy` - How to handle duplicates: UNION_DEDUP (default), LATEST_WINS, FIRST_WINS
- `library_type` - Optional filter (empty = process ALL library types)
- `data_provider_name` - Optional filter (empty = process ALL vendors)
- `data_stream_type` - Optional filter (empty = process ALL streams)

**Result:**
New DTA Template version per (vendor, stream) combination, DTAs marked as `PROMOTED`.


In [ ]:
# Cell 1: Imports
import json
from clinical_data_standards_framework.versioning import (
    merge_dtas_to_template,
    get_version,
    get_all_versions,
    get_current_template_for_vendor_stream
)
from clinical_data_standards_framework.utils import log_dta_version_event

print("="*80)
print("CREATE DTA TEMPLATES")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Config: {len(library_tables)} library table(s)")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters (from job base_parameters)
# This notebook runs as part of job_cdm_version_manager
dbutils.widgets.text("merge_strategy", "UNION_DEDUP", "Merge Strategy")
dbutils.widgets.text("library_type", "", "Library Type (empty=ALL)")
dbutils.widgets.text("data_provider_name", "", "Vendor Filter (empty=ALL)")
dbutils.widgets.text("data_stream_type", "", "Stream Filter (empty=ALL)")

merge_strategy = dbutils.widgets.get("merge_strategy")
library_type_param = dbutils.widgets.get("library_type")
vendor_filter = dbutils.widgets.get("data_provider_name")
stream_filter = dbutils.widgets.get("data_stream_type")

# Determine which library types to process
# If library_type is specified, only process that type
# If empty, process ALL configured library types
if library_type_param:
    library_types_to_process = [lib for lib in library_tables if lib.get('library_type') == library_type_param]
    if not library_types_to_process:
        raise ValueError(
            f"library_type '{library_type_param}' not found in config. "
            f"Available types: {[lib.get('library_type') for lib in library_tables]}"
        )
else:
    # Process ALL configured library types
    if not library_tables:
        raise ValueError(
            "No library_tables configured in versioning config. "
            "Check that md_config_cache is populated correctly."
        )
    library_types_to_process = library_tables

print(f"\nMerge Strategy: {merge_strategy}")
print(f"Library types to process: {[lib.get('library_type') for lib in library_types_to_process]}")
print(f"Vendor Filter: {vendor_filter or '(all vendors)'}")
print(f"Stream Filter: {stream_filter or '(all streams)'}")


In [ ]:
# Cell 4: Configure Table Names
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

# Build library table map from config
library_table_map = {}
for lib_config in library_types_to_process:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

print(f"\nRegistry Table: {registry_table}")
print(f"Library Tables:")
for lib_type, table in library_table_map.items():
    print(f"  {lib_type}: {table}")


In [ ]:
# Cell 5: Query Active DTA Approved Versions Grouped by (Vendor, Stream)
print("\n" + "="*80)
print("Finding Approved DTAs Grouped by Vendor + Stream")
print("="*80)

dta_table = f"{catalog}.{gold_schema}.dta"

# =============================================================================
# Group DTAs by (library_type, data_provider_name, data_stream_type)
# Each group will create its own DTA Template
# =============================================================================

# Structure: {(library_type, vendor, stream): {'versions': [...], 'dta_ids': [...]}}
dta_groups = {}

# Track all DTAs being promoted (for status update at end)
all_promoted_dta_ids = set()

for library_type, library_table in library_table_map.items():
    print(f"\n[{library_type}]")
    
    # Build query with optional vendor/stream filters
    filter_clause = ""
    if vendor_filter:
        filter_clause += f" AND d.data_provider_name = '{vendor_filter}'"
    if stream_filter:
        filter_clause += f" AND d.data_stream_type = '{stream_filter}'"
    
    # Get approved DTAs NOT yet promoted, grouped by vendor+stream
    df_active_dtas = spark.sql(f"""
        SELECT 
            r.version, 
            r.dta_id, 
            r.record_count, 
            r.created_ts, 
            d.dta_number, 
            d.data_provider_name,
            d.data_stream_type,
            d.status as dta_status
        FROM {registry_table} r
        JOIN {dta_table} d ON r.dta_id = d.dta_id
        WHERE r.version_type = 'DTA_APPROVED'
          AND r.status = 'ACTIVE'
          AND r.library_type = '{library_type}'
          AND d.status != 'PROMOTED'
          {filter_clause}
        ORDER BY d.data_provider_name, d.data_stream_type, r.created_ts DESC
    """)
    
    rows = df_active_dtas.collect()
    
    if not rows:
        print(f"  No approved DTAs ready for template creation")
        continue
    
    # Group by (vendor, stream)
    for row in rows:
        key = (library_type, row.data_provider_name, row.data_stream_type)
        if key not in dta_groups:
            dta_groups[key] = {'versions': [], 'dta_ids': [], 'dta_numbers': []}
        dta_groups[key]['versions'].append(row.version)
        dta_groups[key]['dta_ids'].append(row.dta_id)
        dta_groups[key]['dta_numbers'].append(row.dta_number)
        all_promoted_dta_ids.add(row.dta_id)
    
    # Display what we found
    df_active_dtas.show(truncate=False)

# Summary of groups
print(f"\n{'='*60}")
print(f"Summary: {len(dta_groups)} vendor+stream group(s) to process")
print(f"{'='*60}")
for (lib_type, vendor, stream), group_data in dta_groups.items():
    print(f"  [{lib_type}] {vendor} / {stream}: {len(group_data['versions'])} DTA(s)")
    for dta_num in group_data['dta_numbers'][:3]:
        print(f"    - {dta_num}")
    if len(group_data['dta_numbers']) > 3:
        print(f"    ... and {len(group_data['dta_numbers']) - 3} more")

if not dta_groups:
    raise ValueError(
        "No approved DTAs found to create templates.\n"
        "All DTAs are either already PROMOTED or no DTA_APPROVED versions exist.\n"
        "Run job_cdm_dta_import_test first to create DTAs with DTA_APPROVED versions."
    )

print(f"\n✓ Total vendor+stream groups to process: {len(dta_groups)}")


In [ ]:
# Cell 6: Show Current Templates per Vendor+Stream
print("\n" + "="*80)
print("Current DTA Templates by Vendor+Stream")
print("="*80)

# Show existing templates for each vendor+stream we're about to process
for (lib_type, vendor, stream), group_data in dta_groups.items():
    print(f"\n[{lib_type}] {vendor} / {stream}")
    
    current_template = get_current_template_for_vendor_stream(
        spark, registry_table, lib_type, vendor, stream
    )
    
    if current_template:
        print(f"  Current template: v{current_template['version']} ({current_template['record_count']} records)")
        print(f"  Previously included DTAs: {current_template['included_dta_ids']}")
    else:
        print(f"  No template exists yet (first creation for this vendor+stream)")


In [ ]:
# Cell 7: Create DTA Templates per Vendor+Stream
print("\n" + "="*80)
print("Creating DTA Templates per Vendor+Stream")
print("="*80)

# Track new templates created: {(lib_type, vendor, stream): version}
new_templates = {}

for (lib_type, vendor, stream), group_data in dta_groups.items():
    dta_versions = group_data['versions']
    dta_ids = group_data['dta_ids']
    
    # Get library table for this library type
    library_table = library_table_map.get(lib_type)
    if not library_table:
        print(f"\n⚠️ No library table configured for {lib_type}, skipping...")
        continue
    
    print(f"\n[{lib_type}] {vendor} / {stream}")
    print(f"  Merging {len(dta_versions)} DTA(s) -> DTA Template")
    
    new_version, template_dta_id = merge_dtas_to_template(
        spark=spark,
        library_table=library_table,
        registry_table=registry_table,
        dta_table=dta_table,
        dta_versions=dta_versions,
        library_type=lib_type,
        data_provider_name=vendor,
        data_stream_type=stream,
        dta_ids=dta_ids,
        merge_strategy=merge_strategy,
        created_by_principal=created_by_principal,
        databricks_job_id=databricks_job_id,
        databricks_job_name=databricks_job_name,
        databricks_run_id=databricks_run_id
    )
    
    new_templates[(lib_type, vendor, stream)] = {"version": new_version, "template_dta_id": template_dta_id}
    print(f"  ✅ New DTA Template: v{new_version}")
    
    # Log activity for each DTA that contributed to this template
    for dta_id, dta_version in zip(dta_ids, dta_versions):
        log_dta_version_event(
            spark=spark,
            catalog=catalog,
            dta_id=dta_id,
            activity_type="PROMOTED_TO_TEMPLATE",
            version=new_version,
            performed_by=created_by_principal,
            parent_version=dta_version
        )
    
    print(f"  ✓ Activity logged for {len(dta_ids)} DTA(s)")

# Update DTA status to 'PROMOTED' for all promoted DTAs
# This marks them as included in a DTA Template
if all_promoted_dta_ids:
    dta_ids_str = ", ".join([f"'{dta_id}'" for dta_id in all_promoted_dta_ids])
    spark.sql(f"""
        UPDATE {dta_table}
        SET status = 'PROMOTED',
            last_updated_by_principal = '{created_by_principal}',
            last_updated_ts = current_timestamp()
        WHERE dta_id IN ({dta_ids_str})
    """)
    print(f"\n✓ Updated {len(all_promoted_dta_ids)} DTA(s) to status='PROMOTED'")


In [ ]:
# Cell 8: Set Task Values and Summary

# Serialize new_templates for task values (convert tuple keys to strings)
templates_serializable = {f"{k[0]}|{k[1]}|{k[2]}": v for k, v in new_templates.items()}
groups_serializable = {f"{k[0]}|{k[1]}|{k[2]}": v for k, v in dta_groups.items()}

dbutils.jobs.taskValues.set(key="new_templates", value=json.dumps(templates_serializable))
dbutils.jobs.taskValues.set(key="dta_groups", value=json.dumps(groups_serializable))
dbutils.jobs.taskValues.set(key="merge_strategy", value=merge_strategy)

print("\n" + "="*80)
print("DTA Template History")
print("="*80)

# Show template history for each vendor+stream processed
for (lib_type, vendor, stream), template_info in new_templates.items():
    new_version = template_info["version"]
    template_dta_id = template_info["template_dta_id"]
    print(f"\n[{lib_type}] {vendor} / {stream}")
    print(f"  Template DTA ID: {template_dta_id}")
    try:
        df_templates = spark.sql(f"""
            SELECT version, status, record_count, data_provider_name, data_stream_type, dta_id, created_ts
            FROM {registry_table}
            WHERE version_type = 'DTA_TEMPLATE'
              AND library_type = '{lib_type}'
              AND data_provider_name = '{vendor}'
              AND data_stream_type = '{stream}'
            ORDER BY created_ts DESC
            LIMIT 5
        """)
        df_templates.show(truncate=False)
    except Exception as e:
        print(f"  Error retrieving versions: {e}")

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Strategy: {merge_strategy}")
print(f"Templates Created: {len(new_templates)}")
print(f"DTAs Promoted: {len(all_promoted_dta_ids)}")

for (lib_type, vendor, stream), template_info in new_templates.items():
    new_version = template_info["version"]
    template_dta_id = template_info["template_dta_id"]
    group_data = dta_groups.get((lib_type, vendor, stream), {})
    dta_numbers = group_data.get('dta_numbers', [])
    print(f"\n{lib_type} | {vendor} | {stream}:")
    print(f"  New Template Version: v{new_version}")
    print(f"  Template DTA ID: {template_dta_id}")
    print(f"  Merged from {len(dta_numbers)} DTA(s):")
    for dta_num in dta_numbers[:5]:
        print(f"    - {dta_num}")
    if len(dta_numbers) > 5:
        print(f"    ... and {len(dta_numbers) - 5} more")

print(f"\n✅ DTA Template creation completed successfully!")
